# Import Packages and Mount Drive 

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob, json, zipfile, os, csv, re, nltk

import pandas as pd
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import copy as cp
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('vader_lexicon')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm.notebook import tqdm as tq
tq.pandas()
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LdaMulticore
from gensim.utils import simple_preprocess 
import gensim.corpora as corpora
from google.colab import files
analyzer = SentimentIntensityAnalyzer()




/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Set path to local directory 

## Initalize Paths

In [ ]:
CHECKPOINT_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/checkpoints'
BATCH_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/batched_data'
DATA_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /Preprocessing/data_to_process/temp'
KEYWORDS_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /Preprocessing/COVID_keywords.txt'
RESULTS_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2020June16'

## Initialize Checkpoint I/O

In [ ]:

def write_json(filename, df, path):
  
  with open(path + filename, 'w') as f: 
      result = df.to_json(orient = 'records', double_precision = 15)
      parsed = json.loads(result)
      f.write(json.dumps(parsed))
  f.close()

def read_json(filename, path):
  
  with open(path + '/'+ filename, 'r') as f: 
    read = pd.read_json(f, 
                        orient='records', keep_default_dates=False, precise_float=True) 
  return pd.DataFrame(read)

def read_batch(path):

  with open(path, 'r') as f: 
    read = pd.read_json(f, 
                        orient='records', keep_default_dates=False, precise_float=True) 
  return pd.DataFrame(read)

def write_csv(filename, lst, path):

  with open(path + '/' + filename, 'a') as f:

      write = csv.writer(f)
      write.writerows(lst)

def write_dict(filename, dct, path):
  
  with open(path + '/' + filename, 'w') as f:
    writer = csv.writer(f)
    for k,v in dct.items():
      writer.writerow([k,v])

def read_dict(filename, path):
  d = {}
  with open(path + '/' + filename) as f:
    reader = csv.reader(f)
    for k,v in reader: 
      v = v.strip('][').split(', ')
      for i in range(len(v)):
        v[i] = v[i].strip('"')
      d[k] = v

  return d


#Load Batched Data

In [ ]:
pd.set_option('precision', 30)
# Set dates to batch 
batches_to_merge = sorted(list(glob.iglob(BATCH_PATH + '/*.json')))

frames = []
total0 = len(batches_to_merge)
pbar0 = tq(total=total0, position = 0, leave = True)
for batch in batches_to_merge:
  filename = 'connected_tweets_' + date + '.json'
  df = read_batch(batch)
  frames.append(df)
  pbar0.update(1)
pbar0.close()
connected_df = pd.concat(frames)
print(connected_df.shape)
connected_df.reset_index(inplace=True)
connected_df = connected_df.drop_duplicates(subset=['tweet-id'])
connected_df = connected_df.where(pd.notnull(connected_df), None)

print(connected_df.shape)

print("Successfully loaded all batched data into dataframe.")


(631460, 22)
(613949, 23)
Successfully loaded all batched data into dataframe.


In [ ]:
out_of_batch_replies = read_dict('out_of_batch_replies_20210616.csv', CHECKPOINT_PATH)
out_of_batch_quotes = read_dict('out_of_batch_quotes_20210616.csv', CHECKPOINT_PATH)

In [ ]:
for key in out_of_batch_replies.keys():
  index = connected_df.index[connected_df['tweet-id'] == int(key)].to_list()
  if index: 
    i = index[0]
    cleaned_keys = []
    for id in out_of_batch_replies[key]:
      id = id.strip("[']")
      cleaned_keys.append(id)    
    connected_df.at[i,'replies'] = connected_df.at[i, 'replies'] = [*connected_df.at[i,'replies'], *cleaned_keys]
    
for key in out_of_batch_quotes.keys():
  index = connected_df.index[connected_df['tweet-id'] == int(key)].to_list()
  if index: 
    i = index[0]
    cleaned_keys = []
    for id in out_of_batch_quotes[key]:
      id = id.strip("[']")
      cleaned_keys.append(id)    
    connected_df.at[i,'quotes'] = connected_df.at[i, 'quotes'] = [*connected_df.at[i,'quotes'], *cleaned_keys]
  

# Preprocess Text

**Steps of preprocessing:**

1.   Remove non-alphanumeric characters
2.   Remove user mentions and urls
3.   Remove # from hashtags
4.   Tag parts of speech and lemmatize 
5.   Tokenize text 








In [ ]:
# initialize wordset
WORDSET = set()
# download stopwords list from nltk library
stop_words_en = stopwords.words('english')
stop_words_spa = stopwords.words('spanish')
# manually extend stopwards list
stop_words_en.extend(["i'm", "\n", "amp", "it's", "it’s", "im", "via", "like", "day", 
                  "time", "go", "got", "get", "one", "lol", "y'all", "y’all", "gonna", "fuck", 
                  "shit", "pls", "plz", "tbh", "rn", "tbt", "ngl", "idc", "idk", "imy", 
                    "i'll", "i've", "let's", "jk", "cuz", "fucking", "fuckin" "wow", "that's", 
                   "dawg", "drop", "pin", "i'd", "lmao", "wtf", "link", "bio"])

"""
Helper function that removes emoji characters from a given text. 

Parameters: 
  string -- string of text 
Returns: 
  demoji_string -- text without emoji characters

"""
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
 
    demoji_string = emoji_pattern.sub(r'', str(string))
    return demoji_string
    

"""
Helper function that transforms part of speech tag to wordnet tag. 

Parameters: 
  nltk_tag -- nltk part of speech tag
Returns: 
  corresponding wordnet tag
"""
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

"""
Cleans the text of the tweet by removing non-alphanumeric characters and 
stop words, and lemmatizing and tokenizing text. 

Parameters: 
  text -- string of tweet text 

Returns: 
  new_text -- cleaned and tokenized text
"""
def clean_text(text):
  
  # Remove emojis
  text = remove_emoji(text)
  # Remove urls
  text = re.sub(r"https\S+", "", text)
  # Remove mentions
  text = re.sub(r"@\S+", "", text)
  # Remove hashtags
  text = re.sub(r"#\S+", "", text)
  # Remove numbers
  text = re.sub(r"[0-9\-]", "", text)
  # Remove punctuation 
  text = re.sub('[,\/.!?:;_#\-\"\“@&%*($)+]', "",text)
  text = re.sub('\’',"\'", text)
  # Lower case letters 
  text = text.lower()

  # Part of speech tagging
  pos_tag = nltk.pos_tag(text.split())
  wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), pos_tag)
  # Lemmatize
  lemmatizer = WordNetLemmatizer() 
  #stemmer = SnowballStemmer("english") 
  lemmatized_words = []
  
  for word, tag in wordnet_tagged:
      if tag is None: 
        lword = lemmatizer.lemmatize(word)
      else:
        lword = lemmatizer.lemmatize(word, pos=tag)
      #sword = stemmer.stem(lword)
      if (lword not in stop_words_en) and (lword not in stop_words_spa) and (len(lword) > 3):
        lemmatized_words.append(lword)
        WORDSET.add(lword)
      

  new_text = lemmatized_words
  
  return new_text

connected_df_dict = connected_df.to_dict(orient = 'records')
total = len(connected_df_dict)
pbar2 = tq(total=total, position = 0, leave = True)

cleaned_tweets = cp.copy(connected_df_dict)
for tweet in cleaned_tweets:
  tweet['cleaned_text'] = clean_text(cp.copy(tweet['text']))
  pbar2.update(1)


pbar2.close() 



# Create Pandas Dataframe 

Merge connected tweets files if batched.

In [ ]:

# determine dataframe columns 
columns = ['tweet-id', 'user-id', 'user-screen_name', 'verified', 'text', 'cleaned_text', 
           'hashtags', 'mentions' , 'in-reply-to', 'quoted', 'timestamp', 'followers-count',
           'replies_count', 'quote_count', 'replies','quotes']



# Create a DataFrame from `tweets`
cleaned_df = pd.DataFrame(cleaned_tweets, columns=columns)
cleaned_df = cleaned_df.where(pd.notnull(cleaned_df), None)

cleaned_df['hashtags'] = cleaned_df['hashtags'].apply(lambda x: [w.lower() for w in x]) 


In [ ]:
# Hashtag and mentions analysis 
from collections import Counter
all_hashtags = []
for i,v in cleaned_df['hashtags'].items():
  for tag in v:
    all_hashtags.append(tag)
hashtag_count = Counter(all_hashtags)
all_mentions = []
for i,v in cleaned_df['mentions'].items():
  for id in v:
    all_mentions.append(id)
mention_count = Counter(all_mentions)


def count_occurences(entity, counter, thresh):
  new_dict = {}
  for e in entity: 
    count = counter[e]
    if count > thresh:
      new_dict[e] = count
  return new_dict

hashtag_thresh = 10
mention_thresh = 10
cleaned_df['hashtags'] = cleaned_df['hashtags'].apply(lambda x : count_occurences(x, hashtag_count, hashtag_thresh))
cleaned_df['mentions'] = cleaned_df['mentions'].apply(lambda x : count_occurences(x, mention_count, mention_thresh))

In [ ]:
user_info = ['user-id', 'user-screen_name', 'verified', 'followers-count', 
             'favorites-count', 'following-count', 'status-count']
user_counts = ['tweets-in-dataset', 'num-tweet-in-dataset', 'replies-in-data-set', 
               'quotes-in-data-set', 'num-replies', 'num-quotes']


unique_users = pd.unique(cleaned_df['user-id'])
user_array = []
for user in unique_users:
  user_data = [] 
  index = connected_df.index[connected_df['user-id'] == user].to_list()[0]
  for header in user_info: 
    attribute = connected_df.at[index, header]
    user_data.append(attribute)
  user_df = connected_df.loc[connected_df['user-id'] == user]
  tweets = user_df['tweet-id'].to_list()
  tweet_count = user_df.shape[0]
  replies = []
  quotes = []
  for i, v in user_df.iterrows():
    replies = [*replies, *v['replies']] 
    quotes = [*quotes, *v['quotes']]
  num_replies = len(replies)
  num_quotes = len(quotes)
  user_data = [*user_data, *[tweets, tweet_count, replies, quotes, num_replies, num_quotes]]
  user_array.append(user_data)

user_df = pd.DataFrame(user_array, columns=[*user_info, *user_counts])


In [ ]:
write_json('processed_tweets_20mar11-21june16.json', cleaned_df, RESULTS_PATH)
write_json('user_data_20mar11-21june16.json', user_df, RESULTS_PATH)
print("Done")

Done


Checkpoint #2

In [ ]:
write_json('cleaned_tweets.json', cleaned_df, CHECKPOINT_PATH)
print("Successfully Saved Checkpoint #2.")

# Keyword Extraction using TFIDF


In [ ]:
cleaned_df = read_json('cleaned_tweets.json')

Extracts top $n$ keywords for each tweet. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from operator import itemgetter

docs = cp.copy(cleaned_df)['cleaned_text'].to_list()
all_text = []
for tweet in docs:
  for word in tweet:
    all_text.append(word)

id2word = corpora.Dictionary(docs)
# Ceate Corpus
texts = docs

corpus = [id2word.doc2bow(text) for text in texts]

vectorizer = TfidfVectorizer()

for i in range(len(texts)):
  texts[i] = ' '.join(texts[i])

tfidf = vectorizer.fit_transform(texts)

feature_names = vectorizer.get_feature_names()

tc = tfidf.tocoo()

doc_dict = {}
for w,d,s in zip(tc.col, tc.row, tc.data):
  word_id = w
  word = feature_names[word_id]
  doc_num = d
  score = s 
  if doc_num in doc_dict.keys():
    doc_dict[doc_num][word] = score
  else:
    doc_dict[doc_num] = {word: score} 

tweets_df = cp.copy(cleaned_df)
tweets_df['keywords'] = tweets_df.apply(lambda x: [], axis=1)


N = 3
all_keywords = []
for doc in doc_dict.keys():
  res = dict(sorted(doc_dict[doc].items(), key = itemgetter(1), reverse = True)[:N])

  keywords = list(res.keys())

  all_keywords.append(keywords)
  tweets_df.at[doc, 'keywords']= keywords


Create Checkpoint #3

In [ ]:
write_json('tweets_keywords.json', tweets_df, CHECKPOINT_PATH)
print("Successfully Saved Checkpoint #3")

# Calculate Sentiment


It first breaks up sentences into clauses (using punctuation and complementizers as separators), and then for each keyword, its score is the average of the vader score of all clauses where it appears.

In [ ]:
def vaderScore(text):
  return analyzer.polarity_scores(text)['compound']

def posTagging(text):
  return nltk.pos_tag(nltk.word_tokenize(text))

def avg(list): 
    return sum(list)/len(list) if len(list)!=0 else 0

clauseDelimeters = ['CC','.','!','?',','] # complementizers and punctuations. 

def sentimentScore(text, aspects):
  tokens = posTagging(text)
  scores = {word: [] for word in aspects} 
      # ALT: just sum scores and divide by total number of clauses
  
  clause = []
  flag = False

  for (word,pos) in tokens:
    # print(pos)
    if pos in clauseDelimeters: # updated logic to include punctuation at the end
      flag = True
    elif flag: # the first word after a delimeter, i.e. first word of new clause
      flag = False
      # process the clause
      clauseText = ' '.join(clause)
      # print(clauseText)
      clauseScore = vaderScore(clauseText)
      for aspect in aspects:
        if aspect in clause:
          scores[aspect].append(clauseScore)
      clause = []
    else:
      flag = False

    # clause.append(word)
    if pos != 'CC':
      clause.append(word) # exclude complementizers but include punctuations

  if len(clause)!=0: # handle the last clause--end of sentence
    clauseText = ' '.join(clause)
    # print(clauseText)
    clauseScore = vaderScore(clauseText)
    for aspect in aspects:
      if aspect in clause:
        scores[aspect].append(clauseScore) 
    
  return {word: avg(scores[word]) for word in aspects}


print("----------Calculating Vader Sentiment Scores-----------")
tweets_df['sentiment_score'] = tweets_df.progress_apply(lambda x : vaderScore(x['text']), axis=1)
print("----------Calculating ASBA Sentiment Dictionaries----------")
tweets_df['sentiment_dict'] = tweets_df.progress_apply(lambda x : sentimentScore(x['text'], x['keywords']), axis=1)


Create Checkpoint #4


In [ ]:
write_json('sentiment_cp,json', tweets_df, CHECKPOINT_PATH)
print("Successfully Saved Checkpoint #4")

# Topic Modelling



## Build LDA Model


Creates corpus in bag-of-words format

In [ ]:
# Create set of words from all tweets
docs = tweets_df['cleaned_text'].to_list()
all_text = []
for tweet in docs:
  for word in tweet:
    all_text.append(word)

# Create Dictionary 
id2word = corpora.Dictionary(docs)

# Ceate Corpus
texts = docs

# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]


Builds latend Dirichlet allocation model

In [ ]:
import gensim
from gensim.models import LdaMulticore, CoherenceModel

a = 0.005 # alpha hyperparameter
b = 0.81 # beta hyperparameter
k = 20 # num_topics hyperparameter

lda_model = gensim.models.LdaMulticore(corpus = corpus, 
                                        id2word=id2word,
                                        num_topics = k, 
                                        chunksize = 1000, 
                                        random_state = 100,
                                        passes = 10, 
                                        alpha = a, 
                                        eta = b)

coherence_model_lda = CoherenceModel(model = lda_model, texts = texts, 
                                       dictionary = id2word, coherence = 'c_v' )
coherence = coherence_model_lda.get_coherence()
print("Coherence Score of LDA model:")
print(coherence)

## Store LDA Model Results 



Creates new pandas data frame for LDA generated topics. Ouputs keywords generated from model for a particular topic with weightings. Weightings are generated from term-topic matrix learned during inference.

In [ ]:
results = []
topic_keywords = []

for i in range(k):
  topic_dict = {}
  cur_topic = lda_model.show_topic(i)
  words = []
  weights = []
  for tup in cur_topic:
    words.append(tup[0])
    weights.append(tup[1])
  topic_dict['topic_id'] = i
  topic_dict['keywords'] = words
  topic_dict['weightings'] = weights
  topic_dict['keywords_weightings'] = cur_topic
  topic_keywords.append(["Topic " + str(i), *words])
  results.append(topic_dict)


write_csv('LDA_topic_keywords.csv', topic_keywords, REULTS_PATH)


topic_df = pd.DataFrame(results)


## Cluster Documents

### LDA Topic Distribution

Classifies documents into most probable learned topic based on topic distribution for given document. Also stores the topic distribution for each topic

In [ ]:
# set min_probability threshold 
thresh = 0.05

topic_nums = []
topic_dist = []
for d in docs:
  # transform docuemtn into bow corpus
  bow = id2word.doc2bow(d)
  # retrieve topic distribution for document
  t = lda_model.get_document_topics(bow)
  prob = []
  for i in t:
    prob.append(i[1])
  # find topic with highest probability from distribution
  topic = prob.index(max(prob))
  N = 3
  topics = dict(sorted(dict(t).items(), key = itemgetter(1), reverse = True)[:N]).keys()
  topic_nums.append(list(topics))
  topic_dist.append(t)
  


tweets_df['LDA_topic'] = topic_nums
tweets_df['LDA_distribution'] = topic_dist


In [ ]:
print(tweets_df.shape)

In [ ]:
cluster = {}
for i in range(20):
  cluster[i] = 0
for i,v in tweets_df['LDA_topic'].items():
  for t in v:
    cluster[t] += 1
print(cluster)

In [ ]:
print(tweets_df['LDA_topic'].head(20))

### $K$-means Clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

TFIDF Transformation


In [ ]:
vectorizer = TfidfVectorizer()
for i in range(len(texts)):
  texts[i] = ' '.join(texts[i])
tfidf = vectorizer.fit_transform(texts)

Build $k$-means model

In [ ]:
kmeans = KMeans(n_clusters=20).fit(tfidf)
kmean_predict = kmeans.fit_predict(tfidf)

Store $k$-means data in topic dataframe.

In [ ]:
kmean_topics = []
for t in kmean_predict:
  kmean_topics.append(t)
tweets_df['kmeans_topic'] = kmean_topics


# Export Data


In [ ]:
"""
Function that writes dataframe to .json file.

Parameters: 
  filename -- output file name
  df -- dataframe to export 
   
"""


output_columns = ['tweet-id', 'text', 'cleaned_text', 'user-id', 'in-reply-to', 'quoted', 
                  'timestamp', 'followers-count', 'replies', 'quotes ', 
                  'quote_count', 'replies_count', 'keywords', 'sentiment_dict', 
                  'sentiment_score', 'LDA_topic', 'LDA_distribution', 'hashtags', 'mentions'] 

document_df = tweets_df[output_columns]

write_json('document_data_test2.json', document_df, RESULTS_PATH)
print("Document Data Successfully Written to Google Drive.")
write_json('topic_data_test2.json', topic_df, RESULTS_PATH)
print("Topic Data Successfully Written to Google Drive.")

In [ ]:
print(document_df['LDA_topic'].head(20))

In [ ]:
CHECKPOINT_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /results/Mar16-Apr16'
df = read_json('document_data_mar16-apr16.json')

In [ ]:
with open('/content/drive/MyDrive/UCLA REU 2021 KG /results/paths_2.json', 'r') as f:
  thread = f.readlines()[8]


In [ ]:
print(thread)

In [ ]:
thread = ['1241466302398484480', '1241468184760184832', '1241469151085907968', '1241469473166462976', '1241469808463327232', '1241470575572221952', '1241471366253047808']



In [ ]:
pd.set_option('max_columns', None)
for t in thread: 
  #print(numpy.int64(t))
  series = df.loc[df['tweet-id'] ==  numpy.int64(t)]
  print(series['tweet-id'])
  print(series.to_dict(orient = 'records'))

In [ ]:
pd.set_option('max_rows', None)
keywords = topic_df['keywords'].to_list()
for i in range(len(keywords)):
  print(keywords[i])